<a href="https://colab.research.google.com/github/GradyRoberts/CS4501-ML/blob/master/CS4501ML_All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4VA - Team Elzey
# Charlottesville Real Estate Valuations

In [0]:
%%capture
%matplotlib inline
!pip install category_encoders
import category_encoders as ce
from sklearn.model_selection import train_test_split, cross_val_score
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from datetime import date

## Data acquisition and preprocessing

### Load datasets from Github

In [0]:
# Points
Points_URL = 'https://raw.githubusercontent.com/GradyRoberts/CS4501-ML/master/Parcel_Owner_Points.csv'
points = pd.read_csv(Points_URL)

# Parcels
Parcels_URL = 'https://raw.githubusercontent.com/GradyRoberts/CS4501-ML/master/Parcel_Area_Details.csv'
parcels = pd.read_csv(Parcels_URL)

# Sales
Real_Estate_Sales_URL = 'https://raw.githubusercontent.com/GradyRoberts/CS4501-ML/master/Real_Estate_Sales.csv'
sales = pd.read_csv(Real_Estate_Sales_URL)

### Get GPS coords and sale history

Merge the X and Y (Lat and Long) columns of the points data into the other datasets. Merge the sales value into the parcels dataset.

In [0]:
# Drop unused columns
points = points.drop(['OBJECTID','FileType','GeoParcelIdentificationNumber',
                      'LegalDescription','LotSquareFeet','OwnerName','ParcelMap',
                      'StreetName','StreetNumber','Unit','UseCode','ZipCode',
                      'Zone'], axis=1)

sales = sales.drop(['RecordID_Int','StreetName','StreetNumber','Unit'], axis=1)

parcels = parcels.drop(['FileType','GeoParcelIdentificationNumber',
                        'IsMultiParcelPolygon','Label','LegalDescription',
                        'MapPage','ModifiedDate','OwnerName','OwnerAddress',
                        'OwnerCityState','OwnerZipCode','StreetName',
                        'StreetNumber','TaxYear','Text','Unit','OBJECTID'], axis=1)


# Convert ParcelNumber to string
points['ParcelNumber'] = points['ParcelNumber'].astype(str)
parcels['ParcelNumber'] = parcels['ParcelNumber'].astype(str)
sales['ParcelNumber'] = sales['ParcelNumber'].astype(str)

# Add SaleAmount, SaleDate, X, and Y to parcels
sales = pd.merge(left=sales, right=points, on='ParcelNumber', sort=False)
parcels = pd.merge(left=parcels, right=sales, on='ParcelNumber', sort=False)

# Drop parcel number
parcels = parcels.drop('ParcelNumber', axis=1)

#numeric Date
k = date(1900, 1, 1)
parcels['Date'] = [(date(*[int(i) for i in x[:10].split("-")]) - k).days for x in parcels['SaleDate']]

# Format the SaleDate (YYYY) and drop all data before 2000
parcels.SaleDate = parcels.SaleDate.str.split('-').str.get(0)
parcels.SaleDate = parcels.SaleDate.astype(int)
parcels = parcels[parcels.SaleDate >= 2000]

# Drop rows that are missing zoning and stage SaleAmount and LotSquareFeet
# for imputation. Drop parcels that were assessed for $0 or $100.
parcels.SaleAmount = parcels.SaleAmount.replace(0, np.NaN)
parcels.LotSquareFeet = parcels.LotSquareFeet.replace(0, np.NaN)
parcels.Assessment = parcels.Assessment.replace(0, np.NaN)
parcels.Assessment = parcels.Assessment.replace(100, np.NaN)
parcels = parcels.dropna(subset=['Zoning','Assessment'])

# Place a cap on Assessment
#mask = parcels.Assessment > 1000000
#parcels.loc[mask, 'Assessment'] = 1000000
#np.sum(mask)/parcels.shape[0] #show ~6% of homes above that value


# Shuffle data
parcels = parcels.sample(frac=1, random_state=42).reset_index(drop=True)
parcels.info()

#It would be easier in the future to keep track of what columns are numeric vs. Catagorical for visualizations
numeric_columns = ['LotSquareFeet', 'Date', 'SaleAmount', 'X','Y']#intentionally excluding 'Assesment'
encoded_columns = ['SaleDate', 'Zoning']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25826 entries, 0 to 25825
Data columns (total 7 columns):
Assessment       25826 non-null float64
LotSquareFeet    17989 non-null float64
Zoning           25826 non-null object
SaleDate         25826 non-null int64
SaleAmount       15692 non-null float64
X                25826 non-null float64
Y                25826 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 1.4+ MB


### OneHotEncoding and Scaling

I was having an issue using a ColumnTransformer pipeline so I am opting to just do the operations separately.

In [0]:
# OneHot with get_dummies
parcels = pd.get_dummies(parcels, columns=['SaleDate', 'Zoning'])

# Save col names and indices
col_names = parcels.columns
inds = list(parcels.index.values)

# Get numeric columns for imputation and scaling
parcels_num = parcels[['LotSquareFeet','SaleAmount']]
parcels = parcels.drop(['LotSquareFeet','SaleAmount'], axis=1)

# Impute
imp = SimpleImputer(strategy='median')
parcels_num = pd.DataFrame(imp.fit_transform(parcels_num), 
                           columns=['LotSquareFeet','SaleAmount'], index=inds)
# Scale
scl = StandardScaler()
parcels_num = pd.DataFrame(scl.fit_transform(parcels_num),
                           columns=['LotSquareFeet','SaleAmount'], index=inds)

parcels = parcels.join(parcels_num)
parcels.head()

,Assessment,X,Y,SaleDate_2000,SaleDate_2001,SaleDate_2002,SaleDate_2003,SaleDate_2004,SaleDate_2005,SaleDate_2006,...,Zoning_URBH,Zoning_WME,Zoning_WMEH,Zoning_WMN,Zoning_WMNH,Zoning_WMW,Zoning_WMWH,Zoning_WSH,LotSquareFeet,SaleAmount
0,282300.0,-78.516656,38.013131,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,-0.022405,-0.129452
1,182400.0,-78.486465,38.017739,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.024305,-0.158043
2,728600.0,-78.475299,38.031103,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.027053,0.102302
3,307500.0,-78.464094,38.025466,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.023574,-0.085208
4,199800.0,-78.491957,38.030280,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.023574,-0.134023


### Split train and test data

In [0]:
parcels_y = parcels['Assessment']
parcels_X = parcels.drop('Assessment', axis=1)

X_train,X_test,y_train,y_test = train_test_split(parcels_X, parcels_y, test_size=0.2, random_state=42)

## Data visualization

In [0]:
from urllib.request import urlopen

#Note that this requires Lat, Long not be standardized/centered
train = X_train.join(y_train)
train['Assessment'] = [i if i < 1000000 else 1000000 for i in train['Assessment']]# a hack for now; should figure out how to 

ax = train.plot(kind="scatter", x="X", y="Y", figsize=(9,7), 
                c="Assessment", colormap=plt.get_cmap("jet"),
                colorbar=True, alpha=0.3)

with urlopen('https://raw.githubusercontent.com/GradyRoberts/CS4501-ML/master/cville.png') as file:
  img = mpimg.imread(file)#, mode='RGB')
  
plt.imshow(img, extent=[-78.525, -78.452, 38.008, 38.072], cmap=plt.get_cmap('jet'))

plt.xlim(-78.525,-78.452)
plt.ylim(38.008,38.072)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Value assessment by parcel in Charlottesville')
plt.show()

NameError: ignored

In [0]:
def reg_train_test_mse(reg, name, X_train, y_train, X_test, y_test):
  reg.fit(X_train, y_train)
  reg_train_pred = reg.predict(X_train)
  reg_train_mse = np.mean([(i-j)**2 for i,j in zip(reg_train_pred, y_train)])
  reg_pred = reg.predict(X_test)
  reg_mse = np.mean([(i-j)**2 for i,j in zip(reg_pred, y_test)])
  if name != '':
    print(name + " Train Mean Squared Error: ", reg_train_mse)
    print(name + " Test Mean Squared Error: ", reg_mse)
  return(reg_train_mse, reg_mse)

In [0]:
#Linear Regression (Normal Equation as Small Dataset)

break#Running the following sections will take a lot of time; want to be able to run all cells for preprossessing with starting these


from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_reg_train_pred = lin_reg.predict(X_train)
lin_reg_train_mse = np.mean([(i-j)**2 for i,j in zip(lin_reg_train_pred, y_train)])
lin_reg_pred = lin_reg.predict(X_test)
lin_reg_mse = np.mean([(i-j)**2 for i,j in zip(lin_reg_pred, y_test)])
print("Linear Regression Train Mean Squared Error:", lin_reg_train_mse)
print("Linear Regression Test Mean Squared Error: ", lin_reg_mse)

#These features are linear in higher dimensions but become destorted in lower dims
for col in numeric_columns:
  k = zip(lin_reg_pred, X_test.loc[:, col], y_test)
  k = sorted(k, key = lambda x: x[1])
  pred, test_col, y_col = zip(*k)
  plt.title(col + ' predictive abilitities')
  plt.plot(test_col, y_col, "b.", alpha = 0.3,)
  plt.plot(test_col, pred, "r-", alpha = 0.3)
  plt.xlim(test_col[100], test_col[-100])
  plt.ylim(0,1000000)
  plt.xlabel(col)
  plt.ylabel("Assessment")
  plt.show()

In [0]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree = 2, include_bias = True)
g = poly_features.fit(X_train)
poly_X_train = g.transform(X_train)
poly_X_test = g.transform(X_test)
poly_reg = LinearRegression()
poly_reg.fit(poly_X_train, y_train)
poly_reg_train_pred = poly_reg.predict(poly_X_train)
poly_reg_train_mse = np.mean([(i-j)**2 for i,j in zip(poly_reg_train_pred, y_train)])
poly_reg_pred = poly_reg.predict(poly_X_test)
poly_reg_mse = np.mean([(i-j)**2 for i,j in zip(poly_reg_pred, y_test)])
print("Polynomial Regression Train Mean Squared Error: ", poly_reg_train_mse)
print("Polynomial Regression Test Mean Squared Error: ", poly_reg_mse)
#way worse for overfitting likely as have all those catagorical vars

In [0]:
#Will try and Regularize the Polynomial Model
from sklearn.linear_model import Ridge, Lasso
poly_ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
reg_train_test_mse(poly_ridge_reg, "Ridge Regularized Polynomial  Regression", X_train, y_train, X_test, y_test)
print("\n")

lasso_ridge_reg = Lasso(alpha = 0.5)#tune hyperparam
reg_train_test_mse(lasso_ridge_reg, "Lasso Regularized Polynomial Regression", X_train, y_train, X_test, y_test)

In [0]:
break #takes absolutely forever; DO NOT RUN
sklearn.svm import SVC
gaussian_kernal = SVC(kernel = "rbf", gamma = 0.1, C=1)
reg_train_test_mse(gaussian_kernal, "Gaussian Kernal ", X_train, y_train, X_test, y_test)
